#        FACE MASK DETECTION 

#### Importing Libraries

In [ ]:
import cv2
import os

#### Number of images in each file

In [3]:
with_mask_files = os.listdir('Train/with_mask')
without_mask_files = os.listdir('Train/without_mask')

In [4]:
print('Number of with mask images:', len(with_mask_files))
print('Number of without mask images:', len(without_mask_files))

Number of with mask images: 644
Number of without mask images: 635


#### Data is balanced in both the classes

#### Converting each image's size to (224,224) 

In [5]:
data=[]
categories=['with_mask','without_mask']
for category in categories:
  path=os.path.join('Train',category)

  label=categories.index(category)

  for file in os.listdir(path):
    img_path=os.path.join(path,file)
    img=cv2.imread(img_path)
    img=cv2.resize(img,(224,224))

    data.append([img,label])

In [6]:
#Length of a data(with_mask+without+mask)
len(data)

1279

In [7]:
#Importing library
import random

In [8]:
#shuffling data to reduce biasness
random.shuffle(data)

In [9]:
#Creating features and labels for each images
X=[]
Y=[]

for features,label in data:
  X.append(features)
  Y.append(label)

In [11]:
print(len(X))
print(len(Y))

1279
1279


In [12]:
#importing library
import numpy as np

In [13]:
#converting the features and labels into numpy array
X=np.array(X)
y=np.array(Y)

In [14]:
print(X.shape)
print(y.shape)

(1279, 224, 224, 3)
(1279,)


#### Normalization

In [15]:
X=X/255

#### train_test_split

In [16]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=10)

#### Importing VGG16 Model

In [17]:
from keras.applications.vgg16 import VGG16

In [18]:
vgg=VGG16()

In [19]:
from keras import Sequential

In [20]:
model=Sequential()

In [21]:
for layer in vgg.layers[:-1]:
  model.add(layer)

In [22]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0         
                                                                 
 block3_conv1 (Conv2D)       (None, 56, 56, 256)       2

In [23]:
#Freezing the VGG16 layers
for layer in model.layers:
  layer.trainable=False

In [24]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0         
                                                                 
 block3_conv1 (Conv2D)       (None, 56, 56, 256)       2

### Adding Output layer

In [25]:
from keras.layers import Dense

In [26]:
model.add(Dense(1,activation='sigmoid'))

In [27]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0         
                                                                 
 block3_conv1 (Conv2D)       (None, 56, 56, 256)       2

In [28]:
model.compile(optimizer='Adam',loss='binary_crossentropy',metrics=['accuracy'])

In [29]:
model.fit(X_train,y_train,epochs=5,validation_data=(X_test,y_test))

Epoch 1/5
32/32 [==============================] - 155s 5s/step - loss: 0.5419 - accuracy: 0.7703 - val_loss: 0.3869 - val_accuracy: 0.8867
Epoch 2/5
32/32 [==============================] - 156s 5s/step - loss: 0.3172 - accuracy: 0.9198 - val_loss: 0.2702 - val_accuracy: 0.9141
Epoch 3/5
32/32 [==============================] - 136s 4s/step - loss: 0.2454 - accuracy: 0.9286 - val_loss: 0.2221 - val_accuracy: 0.9297
Epoch 4/5
32/32 [==============================] - 145s 5s/step - loss: 0.1981 - accuracy: 0.9453 - val_loss: 0.1959 - val_accuracy: 0.9453
Epoch 5/5
32/32 [==============================] - 145s 5s/step - loss: 0.1673 - accuracy: 0.9501 - val_loss: 0.1707 - val_accuracy: 0.9570


In [30]:
#to access the camera
cap=cv2.VideoCapture(0)

In [34]:
while True:
    ret,frame=cap.read()
  #call the detection method
    img=cv2.resize(frame,(224,224))
    y_pred=detect_face_mask(img)
    
    if y_pred==0:
        draw_label(frame,"Mask",(30,30),(0,255,0))
    else:
        draw_label(frame,"No Mask",(30,30),(0,0,255))
        
        
        
    cv2.imshow("window",frame)
        
    if cv2.waitKey(1) & 0xFF==ord('x'):
        break

cv2.destroyAllWindows()

1/1 [==============================] - 0s 254ms/step


1/1 [==============================] - 0s 266ms/step


1/1 [==============================] - 0s 168ms/step


1/1 [==============================] - 0s 161ms/step


1/1 [==============================] - 0s 169ms/step


...

1/1 [==============================] - 1s 1s/step


1/1 [==============================] - 2s 2s/step


1/1 [==============================] - 2s 2s/step


1/1 [==============================] - 1s 1s/step


error: OpenCV(4.6.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\resize.cpp:4052: error: (-215:Assertion failed) !ssize.empty() in function 'cv::resize'


In [31]:
def detect_face_mask(img):
    predictions = model.predict(img.reshape(1, 224, 224, 3))
    
    predicted_class = 1 if predictions[0][0] > 0.5 else 0
    
    return predicted_class

In [32]:
def draw_label(img,text,pos,bg_color):
  text_size=cv2.getTextSize(text,cv2.FONT_HERSHEY_SIMPLEX,1,cv2.FILLED)

  end_x=pos[0]+text_size[0][0]+2
  end_y=pos[1]+text_size[0][0]-2

  cv2.rectangle(img,pos,(end_x,end_y),bg_color,cv2.FILLED)
  cv2.putText(img,text,pos,cv2.FONT_HERSHEY_SIMPLEX,1,(0,0,0,0),1,cv2.LINE_AA)
